In [1]:
import os
import sys
import torch
import numpy as np
import pandas as pd

sys.path.append("..")
from mtecg.utils import load_ecg_dataframe

SEED = 42
np.random.seed(SEED)

c:\Anaconda3\envs\ecg\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Anaconda3\envs\ecg\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
c:\Anaconda3\envs\ecg\lib\site-packages\numpy\.libs\libopenblas64__v0.3.21-gcc_10_3_0.dll
  warnings.warn("loaded more than 1 DLL from .libs:"
c:\Anaconda3\envs\ecg\lib\site-packages\pkg_resources\__init__.py:123: PkgResourcesDeprecationWarning: llow is an invalid version and will not be supported in a future release
  warnings.warn(


In [9]:
# train_dev_csv_path = "../../ECG_EF_Clin_train_dev_new.csv"
# new_test_csv_path = "../../ECG_EF_Clin_test_new_nocut_noimpute.csv"

train_dev_csv_path = "../datasets/all_ECG_cleared_duplicate_may23_final.csv"
new_test_csv_path = "../datasets/all_ECG_cleared_duplicate_may23_final.csv"
imputer_dir = "../trained_models/multi-task-clinical/resnet34d_384_LVEF50_birnn_dim512"

train_dev_image_dir = "../datasets/siriraj_data/ECG_MRI_images_new/"
new_test_image_dir = "../datasets/siriraj_data/ECG_MRI_test_images_new/"

In [11]:
# Old test set.
train_dev_df = load_ecg_dataframe(
    train_dev_csv_path,
    train_dev_image_dir,
    # imputer_dir=imputer_dir,
    do_split=True,
    return_lvef_40_column=True,
)
# New test set. No need to impute.
new_test_df = load_ecg_dataframe(
    new_test_csv_path,
    new_test_image_dir,
    # imputer_dir=imputer_dir,
    do_split=False,
    return_lvef_40_column=True,
)

In [12]:
scar_type_excel_path = "../../AI_ECG_CAD_scar_type_221227.xlsx"
scar_type_df = pd.read_excel(scar_type_excel_path)

# select cols from scar
scar_type_df = scar_type_df[["File_Name","Month","Subendocardial_scar","Transmural_scar","Subendocardial_scar_or_Transmural_scar"]]
# Lowercase column names for consistency.
scar_type_df.columns = map(str.lower, scar_type_df.columns)

In [13]:
# merge scar type onto train dev & test
train_dev_df = pd.merge(train_dev_df, scar_type_df, on=["file_name", "month"], how='left')
new_test_df = pd.merge(new_test_df, scar_type_df, on=["file_name", "month"], how='left')

# remove 2 corrupted PDF
# 2010/10/2010_401658221.pdf
# 2016/6/2016_527006041.pdf
train_dev_df = train_dev_df.drop(train_dev_df[train_dev_df['file_name'].isin(["2010_401658221", "2016_527006041"])].index)

In [19]:
# train_df = train_dev_df[train_dev_df["split"].isin(["old_train", "new_train"]) == 1].reset_index(drop=True)
# dev_df = train_dev_df[train_dev_df["split"].isin(["old_valid", "new_valid"]) == 1].reset_index(drop=True)
# old_test_df = train_dev_df[train_dev_df["split"] == "old_test"].reset_index(drop=True)

old_train_df = train_dev_df[train_dev_df["split"] == "old_train"].reset_index(drop=True)
old_valid_df = train_dev_df[train_dev_df["split"] == "old_valid"].reset_index(drop=True)
old_test_df = train_dev_df[train_dev_df["split"] == "old_test"].reset_index(drop=True)
new_train_df = train_dev_df[train_dev_df["split"] == "new_train"].reset_index(drop=True)
new_valid_df = train_dev_df[train_dev_df["split"] == "new_valid"].reset_index(drop=True)

population_to_df_map_dict = {
    "Training (Old format)": old_train_df,
    "Training (New format)": new_train_df,
    "Development (Old format)": old_valid_df,
    "Development (New format)": new_valid_df,
    "Test (Old format)": old_test_df,
    "Test (New format)": new_test_df,
}

In [20]:
train_df.shape, dev_df.shape, old_test_df.shape, new_test_df.shape

((9393, 34), (2500, 34), (895, 34), (1264, 34))

## **Get stats for the dataset**

In [21]:
def get_n_and_prevalence(dataframe: pd.DataFrame, col: str, inverse: bool = False) -> tuple:
    total_samples = dataframe.shape[0]
    target_columns = [col]
    target_class_index = 0 if inverse else 1

    target_dataframe = dataframe[target_columns].copy()
    n = target_dataframe.value_counts()[target_class_index]
    prevalence = n/total_samples * 100

    n = round(n, 3)
    prevalence = round(prevalence, 3)
    return n, prevalence

# result in % prevalence 
def get_baseline_stats(
    dataframe: pd.DataFrame,
    lvef_col: str = "lvef",
    lvef_40_col: str = "lvef_40",
    scar_col: str = "scar_cad",
    age_col: str = "age",
    female_gender_col: str = "female_gender",
    smoke_col: str = "smoke",
    dm_col: str = "dm",
    ht_col: str = "ht",
    dlp_col: str = "dlp",
    SubS_col: str = "subendocardial_scar",
    TranS_col: str = "transmural_scar",
    ):
    n_samples = dataframe.shape[0]
    # Default values.
    lvef_prevalence = 0
    scar_prevalence = 0
    # Calculate prevalence of LVEF and scar if the columns are present.
    if lvef_col in dataframe.columns:
        lvef_n, lvef_prevalence = get_n_and_prevalence(dataframe, lvef_col)
    if lvef_40_col in dataframe.columns:
        lvef_40_n, lvef_40_prevalence = get_n_and_prevalence(dataframe, lvef_40_col)
    if scar_col in dataframe.columns:
        scar_n, scar_prevalence = get_n_and_prevalence(dataframe, scar_col)

    # Calculate baseline statistics.
    mean_age = dataframe[age_col].mean()
    std_age = dataframe[age_col].std()

    # Scale to back to original values.
    mean_age = round(mean_age * 100, 3) 
    std_age = round(std_age * 100, 3)

    male_n, male_percent = get_n_and_prevalence(dataframe, female_gender_col, inverse=True)
    smoke_n, smoke_percent = get_n_and_prevalence(dataframe, smoke_col)
    ht_n, ht_percent = get_n_and_prevalence(dataframe, ht_col)
    dm_n, dm_percent = get_n_and_prevalence(dataframe, dm_col)
    dlp_n, dlp_percent = get_n_and_prevalence(dataframe, dlp_col)
    SubEn_n, SubEn_prevalence = get_n_and_prevalence(dataframe, SubS_col)
    TranMu_n, TranMu_prevalence = get_n_and_prevalence(dataframe, TranS_col)

    SubEn_and_TranMu_n = dataframe[(dataframe[SubS_col] == 1) & (dataframe[TranS_col] == 1)].shape[0]
    SubEn_and_TranMu_prevalence = SubEn_and_TranMu_n/n_samples * 100
    SubEn_and_TranMu_prevalence = round(SubEn_and_TranMu_prevalence, 3)

    baseline_stat_tuple_dict = {
        "age": [f"{mean_age} +/- {std_age}"],
        "male": [f"{male_n} ({male_percent})"],
        "smoke": [f"{smoke_n} ({smoke_percent})"],
        "ht": [f"{ht_n} ({ht_percent})"],
        "dm": [f"{dm_n} ({dm_percent})"],
        "dlp": [f"{dlp_n} ({dlp_percent})"],
        "scar": [f"{scar_n} ({scar_prevalence})"],
        "lvef": [f"{lvef_n} ({lvef_prevalence})"],
        "lvef_40": [f"{lvef_40_n} ({lvef_40_prevalence})"],
        "SubEn": [f"{SubEn_n} ({SubEn_prevalence})"],
        "TranMu": [f"{TranMu_n} ({TranMu_prevalence})"],
        "SubEn_and_TranMu": [f"{SubEn_and_TranMu_n} ({SubEn_and_TranMu_prevalence})"],
    }

    baseline_stat_dataframe =  pd.DataFrame(baseline_stat_tuple_dict)
    return baseline_stat_dataframe

In [22]:
statistic_dataframe_list = []
for population_name, dataframe in population_to_df_map_dict.items():
    baseline_stat_dataframe = get_baseline_stats(
        dataframe,
        SubS_col = "subs",
        TranS_col = "trans",
        )
    baseline_stat_dataframe.index = [population_name]
    statistic_dataframe_list.append(baseline_stat_dataframe)

all_population_df = pd.concat(population_to_df_map_dict.values(), axis=0)
population_stat_df = get_baseline_stats(
    all_population_df,
    SubS_col = "subs",
    TranS_col = "trans",
    )
population_stat_df.index = ["population"]
statistic_dataframe_list.append(population_stat_df)

In [24]:
pd.concat(statistic_dataframe_list, axis=0)#.to_csv("../resources/statistics/population_statistics_split.csv", index=True)